In [60]:
import awkward as ak
import numpy as np
import coffea
import os
from python.plugins import *

print(coffea.__version__)
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

import hist

# testFiles = [
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_170to300_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/120000/0CDD30E0-B953-E54B-BB7A-BFB45BE6EC93.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_300to470_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/120000/3788BBD3-3B70-BE48-B01A-4FA395E1E0B6.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_470to600_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/F6B6A983-3E2B-7B4D-A2CE-E03AB6E32387.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_600to800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/FDF7FBE4-9226-994A-BD45-29995A3CD62F.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_800to1000_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/2CEB03D7-5B1A-EE46-9C21-13ED3E349AF4.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/DB87A517-0584-1A41-BB18-AAE7C9AAE5B8.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/BFE2E7E9-82CB-FE47-9B43-EBA5C373AF60.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_1800to2400_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/5213B536-BAA1-CC43-A58A-B1E41E2680F5.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_2400to3200_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/E8400F91-3216-EA46-A9F1-8CC7EF5261B0.root",
# "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/B21BFDD8-B878-784C-AFB6-5CDE131F5C39.root"
# ]
# redirector = 'root://xcache/'
# redirector='root://xrootd-local.unl.edu:1094/'
redirector = 'root://cmsxrootd.fnal.gov/'
# files = handleData('fileset_QCD.json', redirector, testing = True, data = False)
# files_test = handleData('fileset_QCD.json', redirector, testing = False, data = False)
# events = NanoEventsFactory.from_root(redirector+"/store/mc/RunIISummer20UL18NanoAODv9/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/280000/C9729B80-15C9-3940-972D-AB1E7610CE6D.root", schemaclass=NanoAODSchema).events()
events = NanoEventsFactory.from_root('root://cmseos.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/QCD_HT700to1000_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2820000/150F2AD2-0267-AE4F-90F9-D8191F29DC95.root', schemaclass=NanoAODSchema).events()
# print(events.fields)
# print("MC prefiring weights ", events['L1PreFiringWeight'].Nom)
events = events[ak.all(events.FatJet[:,:2].pt > 200., axis = -1)]
print(len(events[ak.any(events.FatJet.jetId ==2, axis=-1)]))
print(len(events[ak.any(events.FatJet.jetId ==6, axis=-1)]))
events = events[ak.all(events.GenJetAK8[:,:2].mass < 20., axis=-1) & ak.all(events.FatJet[:,:2].mass >20., axis=-1)]
print(len(events))
print(len(events[ak.any(events.FatJet.jetId ==2, axis=-1)]))
print(events[ak.any(events.FatJet.jetId ==2, axis=-1)].FatJet.mass)
print(events[ak.any(events.FatJet.jetId ==2, axis=-1)].GenJetAK8.mass)
print(len(events[ak.all(events.FatJet.jetId ==6, axis=-1)]))
# print(events.GenJetAK8.mass)
# print(events.FatJet.mass)

# area = events.FatJet[:,:2].area
# print(area)
# rho = events.FatJet[:,:2].mass/events.FatJet[:,:2].pt
# rho_gen = events.GenJetAK8[:,:2].mass/events.GenJetAK8[:,:2].pt
# print("rho max ", np.log(ak.max(rho)), " rho min ", np.log(ak.min(rho)))
# print("rho gen max ", ak.max(np.log(rho_gen)), " rho gen min ", ak.min(np.log(rho_gen)))
# print(rho)



# import uproot
# # ### uproot notation
# # qcd_mc = uproot.open(list(files.values())[-1][0])
# qcd_mc = uproot.open(redirector+"/store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/B21BFDD8-B878-784C-AFB6-5CDE131F5C39.root")
# uprootevents = qcd_mc['Events']
# print(uprootevents.keys('subjet'))

0.7.22
1501
502567
31
1
[[21.3, 31.1]]
[[16.7, 19.9, 27.1]]
30


In [2]:
events = events[ak.all(events.FatJet[:,:2].pt > 200., axis = -1)]
print(events.fields)
print(events.GenJetAK8.fields)

['GenPart', 'SoftActivityJetNjets2', 'HLT', 'L1simulation', 'btagWeight', 'genTtbarId', 'ChsMET', 'GenVisTau', 'Tau', 'SoftActivityJetNjets10', 'Muon', 'run', 'GenMET', 'FsrPhoton', 'GenJet', 'OtherPV', 'FatJet', 'CorrT1METJet', 'GenIsolatedPhoton', 'fixedGridRhoFastjetAll', 'L1', 'PSWeight', 'Generator', 'SoftActivityJetHT10', 'GenJetAK8', 'SoftActivityJetHT', 'TkMET', 'DeepMETResolutionTune', 'LHEWeight', 'Flag', 'fixedGridRhoFastjetCentralNeutral', 'DeepMETResponseTune', 'GenVtx', 'Photon', 'SubJet', 'HLTriggerFirstPath', 'Pileup', 'fixedGridRhoFastjetCentralChargedPileUp', 'CaloMET', 'HLTriggerFinalPath', 'SoftActivityJetNjets5', 'LHEPdfWeight', 'GenDressedLepton', 'HTXS', 'SoftActivityJet', 'RawMET', 'MET', 'LHEPart', 'boostedTau', 'LHEReweightingWeight', 'LHE', 'PuppiMET', 'luminosityBlock', 'TrigObj', 'Jet', 'SubGenJetAK8', 'L1Reco', 'genWeight', 'fixedGridRhoFastjetCentral', 'SoftActivityJetHT2', 'PV', 'IsoTrack', 'RawPuppiMET', 'SoftActivityJetHT5', 'fixedGridRhoFastjetCentral

In [3]:
from python.dijetProcessor import makeDijetHists
# from python.dijetProcessor_old import makeDijetHists
from dask.distributed import Client
import pickle
#### WE'RE MISSING 2016B ver2 -- AK8 PF HLT is missing need to use AK4 trigger isntead
data = False
testing = True
year = "2018"
dask= False
range = None
mctype="pythiaMG"
# mctype="herwig"
winterfell = False
jk= False
if jk:
    data = False
jk_range = [4,6]
unc_srcs = ["nominal","HEM", "JER", "JMR", "JMS", "AbsoluteMPFBias","AbsoluteScale","AbsoluteStat","FlavorQCD","Fragmentation","PileUpDataMC","PileUpPtBB","PileUpPtEC1",
"PileUpPtEC2","PileUpPtHF","PileUpPtRef","RelativeFSR","RelativeJEREC1",
            "RelativeJEREC2","RelativeJERHF","RelativePtBB","RelativePtEC1",
            "RelativePtEC2","RelativePtHF","RelativeBal","RelativeSample",
            "RelativeStatEC","RelativeStatFSR","RelativeStatHF","SinglePionECAL",
            "SinglePionHCAL","TimePtEta"]
if mctype!='pythia':
        unc_srcs.extend(["Q2", "PDF"])
processor = makeDijetHists(data = data, jet_systematics = unc_srcs, jk=jk, jk_range=jk_range) #, systematics = ['LHEPDF']
datastring = "JetHT" if processor.do_gen == False else "QCDsim"
jkstring = ""
if jk:
    if jk_range != None:
        jkstring = "JK" + str(jk_range[0]) + str(jk_range[1])+ "_"
    else:
        jkstring = "JK" 
if year == 2016 or year == 2017 or year == 2018:
    year_str = str(year)
elif year == "2016" or year == "2016APV" or year == "2017" or year == "2018":
    year_str = year
else:
    year_str = "All"
if processor.do_gen==True and winterfell:
    filename = "QCD_flat_files.json"
elif processor.do_gen==True:
    # filename = "fileset_QCD.json"
    if mctype == "pythiaMG":
        filename = "fileset_MG_pythia8_wRedirs.json"
    elif mctype == "herwig":
        filename = "fileset_HERWIG_wRedirs.json"
    else:
        filename = "fileset_QCD_wRedirs.json"
else:
    # filename = "datasets_UL_NANOAOD.json"
    filename = "fileset_JetHT_wRedirs.json"
if testing and not data:
    fname = 'coffeaOutput/dijet/dijetHistsTest_{}_NEWmbindeltaRmatch_{}_{}{}.pkl'.format(datastring, mctype, jkstring,year_str)
elif testing and data:
    fname = 'coffeaOutput/dijet/dijetHistsTest{}_NEWmbindeltaRmatch_{}_{}{}.pkl'.format(datastring, mctype, jkstring,year_str)
elif not testing and data:
    fname = 'coffeaOutput/dijet/dijetHists_{}_{}_NEWmbindeltaRmatch_{}{}.pkl'.format(datastring, mctype,jkstring, year_str)
else:
    fname = 'coffeaOutput/dijet/dijetHists_{}_{}_NEWmbindeltaRmatch_{}{}.pkl'.format(datastring, mctype, jkstring, year_str)
if range!=None:
    print("Range input: ", range)
    fname=fname[:-4]+"_"+str(range[0])+"_"+str(range[1])+".pkl"
    print("New ranged fname ", fname)

0.7.22
2.7.2
Data: Gen:  False True


In [ ]:
run_ind_srcs = False
if run_ind_srcs == True:
    for src in unc_srcs:
        print("Running for source ", src)
        processor = makeDijetHists(data = data, jet_systematics = [src])
        result = runCoffeaJob(processor, jsonFile = filename, year = year, 
                              testing = testing, dask = dask, data = not processor.do_gen, datasetRange=range)
        if testing and not data:
            fname = 'coffeaOutput/dijet/dijetHistsTest_wXSscaling_{}_pt{}_rapidity{}_{}{}.pkl'.format(datastring, processor.ptcut, processor.ycut, src, year_str)
        elif testing and data:
            fname = 'coffeaOutput/dijet/dijetHistsTest{}_pt{}_rapidity{}_{}{}.pkl'.format(datastring, processor.ptcut, processor.ycut, src, year_str)
        elif not testing and data:
            fname = 'coffeaOutput/dijet/dijetHists_{}_pt{}_rapidity{}_{}{}.pkl'.format(datastring, processor.ptcut, processor.ycut, src, year_str)
        else:
            fname = 'coffeaOutput/dijet/dijetHists_wXSscaling_{}_pt{}_rapidity{}_{}{}.pkl'.format(datastring, processor.ptcut, processor.ycut, src, year_str)
        if range!=None:
            print("Range input: ", range)
            fname=fname[:-4]+"_"+str(range[0])+"_"+str(range[1])+".pkl"
            print("New ranged fname ", fname)
        with open(fname, "wb") as f:
            pickle.dump( result, f)
else:
    result = runCoffeaJob(processor, jsonFile = filename, year = year,
                      testing = testing, dask = dask, data = not processor.do_gen)
    with open(fname, "wb") as f:
        pickle.dump( result, f)

Running over datasets  dict_keys(['/QCD_HT700to1000_TuneCP5_13TeV-madgraphMLM-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM'])
Running locally


Output()

Output()

Filename:  root://cmseos.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/QCD_HT700to1000_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2820000/150F2AD2-0267-AE4F-90F9-D8191F29DC95.root
Dataset:  /QCD_HT700to1000_TuneCP5_13TeV-madgraphMLM-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM
pythiaMG2018
Jk indices we'll loop over  range(4, 6)
range max  1
extracting corrections from files for Summer19UL18_V5_MC
N events missing pt entry  25
N events w/ pt entry  106121
Fatjet pt  [[288, 284], [328, 311], [348, 343], ... [352, 335], [300, 233, 175], [412, 412]]
Nevents with negative subjet id  21507  out of  106146
extracting corrections from files for Summer19UL18_V5_MC
N events missing pt entry  474
N events w/ pt entry  105672
Fatjet pt  [[253, 31, 248, 33.6], [185, 97.8, 212, ... 123, 185, 35.9], [243, 183, 202, 167]]
Avail corrected jet objs  ['area', 'btagCSVV2', 'btagDDBvLV2', 'btagDDCvBV2', 'btagDDCvLV2', 'btagDe

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



AK8 sdmass before corr  [[9.34, 58.8], [3.79, 10.5], [7.4, 5.2], ... [24, 35.7, 1.03], [54, 14.9, 1.44]]  and after  [[9.31, 58.8], [3.83, 10.2], [7.25, 4.37, ... [25.2, 35.8, 1.03], [53, 14.9, 1.3]]
Nevents with negative subjet id  21532  out of  106146
extracting corrections from files for Summer19UL18_V5_MC
N events missing pt entry  22
N events w/ pt entry  106124
Fatjet pt  [[398, 312], [324, 239], [308, 247], ... 348], [400, 315, 198], [399, 253, 220]]
Avail corrected jet objs  ['area', 'btagCSVV2', 'btagDDBvLV2', 'btagDDCvBV2', 'btagDDCvLV2', 'btagDeepB', 'btagHbb', 'deepTagMD_H4qvsQCD', 'deepTagMD_HbbvsQCD', 'deepTagMD_TvsQCD', 'deepTagMD_WvsQCD', 'deepTagMD_ZHbbvsQCD', 'deepTagMD_ZHccvsQCD', 'deepTagMD_ZbbvsQCD', 'deepTagMD_ZvsQCD', 'deepTagMD_bbvsLight', 'deepTagMD_ccvsLight', 'deepTag_H', 'deepTag_QCD', 'deepTag_QCDothers', 'deepTag_TvsQCD', 'deepTag_WvsQCD', 'deepTag_ZvsQCD', 'eta', 'mass', 'n2b1', 'n3b1', 'particleNetMD_QCD', 'particleNetMD_Xbb', 'particleNetMD_Xcc', 'part

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



AK8 sdmass before corr  [[31.3, 3.26], [-1, 6.63], [116, 82.5], ... [51.8, 2.7], [74.2, 7.44], [2, 15.2]]  and after  [[32.3, 3.24], [6.76], [118, 81.2], ... [52.6, 2.82], [74.2, 7.43], [1.97, 15.4]]
Nevents with negative subjet id  21896  out of  106146
extracting corrections from files for Summer19UL18_V5_MC
N events missing pt entry  27
N events w/ pt entry  106119
Fatjet pt  [[511, 488], [198, 192], [432, 319], [402, ... [398, 335], [415, 334], [390, 384]]
Avail corrected jet objs  ['area', 'btagCSVV2', 'btagDDBvLV2', 'btagDDCvBV2', 'btagDDCvLV2', 'btagDeepB', 'btagHbb', 'deepTagMD_H4qvsQCD', 'deepTagMD_HbbvsQCD', 'deepTagMD_TvsQCD', 'deepTagMD_WvsQCD', 'deepTagMD_ZHbbvsQCD', 'deepTagMD_ZHccvsQCD', 'deepTagMD_ZbbvsQCD', 'deepTagMD_ZvsQCD', 'deepTagMD_bbvsLight', 'deepTagMD_ccvsLight', 'deepTag_H', 'deepTag_QCD', 'deepTag_QCDothers', 'deepTag_TvsQCD', 'deepTag_WvsQCD', 'deepTag_ZvsQCD', 'eta', 'mass', 'n2b1', 'n3b1', 'particleNetMD_QCD', 'particleNetMD_Xbb', 'particleNetMD_Xcc', 'pa

In [ ]:
with open(fname, "rb") as f:
    result = pickle.load( f )
# result=result[0]
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
%matplotlib inline
# print(result['cutflow'])
plt.rcParams["figure.figsize"] = (10,10)
print(result['jet_eta_phi_precuts'])
df = pd.DataFrame.from_dict(result['cutflow'])
# display(df)
fig, axs = plt.subplots(2, 2)
result['jet_eta_phi'][{'syst':'nominal'}].project("eta", "phi").plot2d(ax=axs[0,0])
result['jet_eta_phi_preveto'][{'syst':'nominal'}].project("eta", "phi").plot2d(ax = axs[0,1])
result['jet_eta_phi_precuts'][{'syst':'nominal'}].project("eta", "phi").plot2d(ax = axs[1,0])
print(result['cutflow']['trigger_init'])
print(result['cutflow']['trigger_final'])

In [ ]:
fig, axs = plt.subplots(2, 2)
fig.suptitle('Ungroomed MC jets')

result['response_matrix_u'][{'syst':"nominal"}].project('ptreco').plot1d(ax=axs[0,0], binwnorm=True)
result['response_rho_u'].project('rhogen').plot1d(ax=axs[0,1])
result['response_rho_u'].project('rhoreco').plot1d(ax=axs[1,0], binwnorm=True)
result['response_matrix_u'].project('mreco').plot1d(ax=axs[1,1])

for ax in axs.flatten():
    ax.set_xlim(0,1000)

In [ ]:
s = result['response_matrix_u'].stack("syst")
s.project('ptreco').plot(binwnorm=True)
plt.legend()
plt.show()

In [ ]:
fname = "coffeaOutput/dijet/dijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5_nominal2018.pkl"
with open(fname, "rb") as f:
    result = pickle.load( f )
# result=result[0]
import matplotlib.pyplot as plt
%matplotlib inline
#import mplhep as hep
plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
fig, axs = plt.subplots(2, 2)
result['ptreco_mreco_u'].project('ptreco').plot1d(ax=axs[0,0], binwnorm=True)
result['ptreco_mreco_u'].project('mreco').plot1d(ax=axs[0,1])
result['ptreco_mreco_g'].project('ptreco').plot1d(ax=axs[1,0], binwnorm=True)
result['ptreco_mreco_g'].project('mreco').plot1d(ax=axs[1,1])

In [ ]:
fig, axs = plt.subplots(2, 2)
for ax in axs.flatten():
    ax.set_xlim([0,1000])
fig.suptitle('Ungroomed Gen jets')
result['ptgen_mgen_u'].project('ptgen').plot1d(ax=axs[0,0], binwnorm=True)
result['ptgen_mgen_u'].project('mgen').plot1d(ax=axs[0,1])
result['ptgen_mgen_g'].project('ptgen').plot1d(ax=axs[1,0], binwnorm=True)
result['ptgen_mgen_g'].project('mgen').plot1d(ax=axs[1,1])

In [ ]:
# numerator = result['jet_pt'][{'partonFlav':'Gluon'}].project('ptreco')
# denominator = result['jet_pt'].project('ptreco')
# print(numerator.values(), denominator.values())

# numerator.label = r'$\epsilon$'
# fig = plt.gcf()
# grid = fig.add_gridspec(2, 1, hspace=0, height_ratios=[3, 1])

# main_ax = fig.add_subplot(grid[0])
# subplot_ax = fig.add_subplot(grid[1], sharex=main_ax)
# plt.setp(main_ax.get_xticklabels(), visible=False)
# main_ax, sublot_ax = numerator.plot_ratio(
#     denominator,
#     rp_num_label="gluons",
#     rp_denom_label="all",
#     rp_uncert_draw_type="line",
#     rp_uncertainty_type="efficiency",
#     ax_dict = {"main_ax":main_ax,"ratio_ax":subplot_ax}
# )

# plt.xlabel("Pt (GeV)")
# plt.ylabel("Gluon fraction")
# fig.suptitle('Gluon content as a function of pT')

In [ ]:
# numerator = result['jet_eta'][{'partonFlav':'Gluon'}].project('eta')
# denominator = result['jet_eta'].project('eta')
# print(numerator.values(), denominator.values())
# numerator.label = r'$\epsilon$'
# numerator.label = r'$\epsilon$'
# fig = plt.gcf()
# grid = fig.add_gridspec(2, 1, hspace=0, height_ratios=[3, 1])

# main_axis = fig.add_subplot(grid[0])
# ratio_ax = fig.add_subplot(grid[1], sharex=main_axis)
# plt.setp(main_axis.get_xticklabels(), visible=False)
# main_ax, sublot_ax = numerator.plot_ratio(
#     denominator,
#     rp_num_label="gluons",
#     rp_denom_label="all",
#     rp_uncert_draw_type="line",
#     rp_uncertainty_type="efficiency",
#     ax_dict = {"main_ax":main_axis,"ratio_ax":ratio_ax}
# )

# plt.xlabel(r'$\eta$')
# plt.ylabel("Gluon fraction")
# main_axis.set_title('Gluon content as a function of eta')


In [ ]:
# #sum entries in bins --> plot as y-axis
# plt.rcParams["figure.figsize"] = (20,5)


# fig, axs = plt.subplots(1, 3)
# result['jet_mass'][{'jetNumb':sum, 'dataset':sum}].plot1d(ax = axs[0], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# plt.xlabel("Jet mass (GeV)")
# result['jet_pt'][{'jetNumb':sum, 'dataset':sum}].plot1d(ax = axs[1], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# #axs[0].set_xlabel("Jet mass (GeV)")
# plt.xlabel("Jet pT (GeV)")
# result['jet_eta'][{'jetNumb':sum, 'dataset':sum}].plot1d(ax = axs[2], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# plt.legend()
# plt.xlabel("Jet eta (GeV)")

# fig.suptitle('Dijet eta <2.5, pt> 200.', fontsize="large")

In [ ]:
# plt.rcParams["figure.figsize"] = (20,10)
# fig, axs = plt.subplots(2, 3)
# print(result['jet_mass'])
# result['jet_mass'][{'jetNumb':'jet1', 'dataset':sum}].plot1d(ax = axs[0,0], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# plt.xlabel("Jet mass (GeV)")
# result['jet_pt'][{'jetNumb':'jet1', 'dataset':sum}].plot1d(ax = axs[0,1], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# #axs[0].set_xlabel("Jet mass (GeV)")
# plt.xlabel("Jet pT (GeV)")
# result['jet_eta'][{'jetNumb':'jet1', 'dataset':sum}].plot1d(ax = axs[0,2], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# plt.legend()
# plt.xlabel("Jet eta (GeV)")
# result['jet_mass'][{'jetNumb':'jet2', 'dataset':sum}].plot1d(ax = axs[1,0], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# plt.xlabel("Jet mass (GeV)")
# result['jet_pt'][{'jetNumb':'jet2', 'dataset':sum}].plot1d(ax = axs[1,1], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# #axs[0].set_xlabel("Jet mass (GeV)")
# plt.xlabel("Jet pT (GeV)")
# result['jet_eta'][{'jetNumb':'jet2', 'dataset':sum}].plot1d(ax = axs[1,2], overlay='partonFlav')
# plt.rc('legend',fontsize='small')
# plt.legend()
# plt.xlabel("Jet eta (GeV)")
# fig.suptitle('Dijet eta <2.5, pt> 200.', fontsize="large")

In [ ]:
# njets=result['cutflow']['nJets']
# ngluonjets=result['cutflow']['nGluonJets']
# print(result['cutflow'])
# print("Gluon purity:", ngluonjets/njets)

In [ ]:
response_matrix_u_values = result['response_matrix_u'].project("ptreco", "mreco", "ptgen", "mgen").values()
response_matrix_g_values = result['response_matrix_g'].project("ptreco", "mreco", "ptgen", "mgen").values()

In [ ]:
nptreco,nmassreco,nptgen,nmassgen = response_matrix_u_values.shape

In [ ]:
response_matrix_u_final = response_matrix_u_values.reshape( (nptreco)*(nmassreco), (nptgen)*(nmassgen) )
print(ak.max(response_matrix_u_final))

In [ ]:
response_matrix_g_final = response_matrix_g_values.reshape( (nptreco)*(nmassreco), (nptgen)*(nmassgen) )

In [ ]:
plt.rcParams["figure.figsize"] = (20,20)
fig, axs = plt.subplots(2, 2)
axs[0,0].imshow( np.log(response_matrix_u_final+1), aspect="auto", cmap="Blues", interpolation='nearest' )
axs[1,1].imshow( np.log(response_matrix_g_final+1), aspect="auto", cmap="Blues", interpolation='nearest' )
plt.xlabel("RECO", fontsize=20)
plt.ylabel("GEN", fontsize=20)
plt.tick_params(labelsize=20)

In [ ]:
plt.figure(figsize = (40,80))
plt.imshow( np.log(response_matrix_g_final+1), aspect="equal", cmap="Blues" )
plt.xlabel("RECO", fontsize=50)
plt.ylabel("GEN", fontsize=50)
plt.tick_params(labelsize=40)

In [ ]:
plt.figure(figsize = (40,80))
plt.imshow( response_matrix_u_final, aspect="equal", cmap="Blues" )
plt.xlabel("RECO", fontsize=50)
plt.ylabel("GEN", fontsize=50)
plt.tick_params(labelsize=40)